In [1]:
pip list

Package                            Version
---------------------------------- --------------------
alabaster                          0.7.12
anaconda-client                    1.9.0
anaconda-navigator                 2.1.1
anaconda-project                   0.10.1
anyio                              2.2.0
appdirs                            1.4.4
applaunchservices                  0.2.1
appnope                            0.1.2
appscript                          1.1.2
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            4.3.1
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              21.2.0
autopep8                           1.5.7
Babel                              2.9.1
backcall                           0.2.0
backports.functools-lru-cache      

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip list

Package                            Version
---------------------------------- --------------------
alabaster                          0.7.12
anaconda-client                    1.9.0
anaconda-navigator                 2.1.1
anaconda-project                   0.10.1
anyio                              2.2.0
appdirs                            1.4.4
applaunchservices                  0.2.1
appnope                            0.1.2
appscript                          1.1.2
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            4.3.1
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              21.2.0
autopep8                           1.5.7
Babel                              2.9.1
backcall                           0.2.0
backports.functools-lru-cache      

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch

In [5]:
#load the libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import variable
import torchvision
import pathlib

In [6]:
#checking the device 
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
print(device)

cpu


In [8]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #0-255 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [9]:
#Dataloader

#Path for training and testing 
train_path = '/Users/gourabsaha/Desktop/scene_detection/seg_train/seg_train'
test_path  = '/Users/gourabsaha/Desktop/scene_detection/seg_test/seg_test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transformer),
    batch_size=256, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transformer),
    batch_size=256, shuffle=True
)

In [10]:
#categories
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [11]:
print (classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [12]:
#CNN Network 
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #output size after convolution filter 
        #((w-f+2p)/s) +1
        #Input Shape = (256,3,150,150)
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape=(256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape=(256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape=(256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image factor by 2 
        #Shape=(256,12,75,75)
        
        #adding another convolutional layer and this time we have imput channel as 12 and output channel as 20
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape=(256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape=(256,20,75,75)
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape=(256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape=(256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape=(256,32,75,75)
        
        #This is a random convolution network to explain how we can add different layers to increase the accuracy of the network
        
        self.fc=nn.Linear(in_features=32*75*75,out_features=num_classes) 
        
        #Feed forwad function
    def forward (self, input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        
        #the above output will be in the matrix form with the shape (256,32,75,75)
            
        output = output.view(-1,32*75*75)
        output = self.fc(output)
            
        return output 

In [13]:
model=ConvNet(num_classes=6).to(device)

In [14]:
#Optmizer and loss function
optimizer= Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [15]:
num_epochs=10

In [16]:
#Calculating the size of training and testing images 
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [17]:
print(train_count,test_count)

14034 3000


In [19]:
# In the final step we will be training the CNN network on the training dataloader and calculate the training loss and accuracy for each epochs and we will save the model for the epoch which will give the best testing accuracy
#Model training and saving the best model

best_accuracy = 0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()# loss.backward will do the backward rotation
        optimizer.step() # This will update the weight and the Bias 
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data, 1)
       
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy = train_accuracy/train_count
    train_accuracy = train_loss/train_count

    #Evaluation on testing dataset
    model.eval()
    
    test_accuracy = 0.0
    
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data, 1)
        test_accuracy+=int (torch. sum(prediction==labels.data))
        
        test_accuracy=test_accuracy/test_count
    
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(int(train_loss)) + ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    #writing the if condition to save the best model
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy = test_accuracy
    

Epoch: 0 Train Loss: 17303 Train Accuracy: tensor(1.2330) Test Accuracy: 0.04201934040967972
Epoch: 1 Train Loss: 9952 Train Accuracy: tensor(0.7091) Test Accuracy: 0.04302056192818596
Epoch: 2 Train Loss: 9896 Train Accuracy: tensor(0.7052) Test Accuracy: 0.04568689578006253
Epoch: 3 Train Loss: 6798 Train Accuracy: tensor(0.4844) Test Accuracy: 0.04368600659464269
Epoch: 4 Train Loss: 5155 Train Accuracy: tensor(0.3673) Test Accuracy: 0.045354562706087224
Epoch: 5 Train Loss: 3350 Train Accuracy: tensor(0.2388) Test Accuracy: 0.04768734033563043
Epoch: 6 Train Loss: 3001 Train Accuracy: tensor(0.2139) Test Accuracy: 0.04668767333571685
Epoch: 7 Train Loss: 2266 Train Accuracy: tensor(0.1615) Test Accuracy: 0.044685451483877356
Epoch: 8 Train Loss: 1509 Train Accuracy: tensor(0.1075) Test Accuracy: 0.04335489655780325
Epoch: 9 Train Loss: 1683 Train Accuracy: tensor(0.1200) Test Accuracy: 0.04502111829865513
